**1.) Importinng the  required modules**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader


**2.) Importing the pre-trained vgg16 model**

In [2]:
vgg16 = models.vgg16(pretrained=True)

c:\Users\Devab\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Devab\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


**3.) Defining the transforms to prepare the dataset for model training**

In [3]:
# Defining the transformations that we need to do to the data present in the dataset 

transform=transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to VGG16 input size
    transforms.RandomHorizontalFlip(),  # Data Augmentation
    transforms.ToTensor(),  # Convert to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])  # ImageNet normalization

**4.) Importing the cifar-100 dataset**

In [4]:
train_data=datasets.CIFAR100(root="./data",train=True,transform=transform,download=True)
test_data=datasets.CIFAR100(root="./data",train=False,transform=transform,download=True)

Files already downloaded and verified
Files already downloaded and verified


**5.) Creating dataloaders to efficiently load the train and test data into the model**

In [5]:
trainloader=DataLoader(train_data,batch_size=32,shuffle=True,num_workers=2)
testloader=DataLoader(test_data,batch_size=32,shuffle=False,num_workers=2)

**6.) Freezing all the convolutional layers (so that we don't alter the core structure of the pre trained model)**

In [6]:
for param in vgg16.features.parameters():
    param.requires_grad = False

**7.) Modifying the vgg16 model to use with cifar100 dataset**

In [7]:
classifier= nn.Sequential(

nn.Linear(25088,4096),
nn.ReLU(),
nn.Dropout(0.5), #Dropout layers are used to prevent overfitting the model 
nn.Linear(4096, 1024),
nn.ReLU(),
nn.Dropout(0.5),
nn.Linear(1024, 100)

)

**8.) Definign the loss function and the optimiser**

In [8]:
crit=nn.CrossEntropyLoss() 

optimiser=optim.Adam(vgg16.classifier.parameters(), lr=0.001)

In [ ]:
epochs = 10  
losses = []

for epoch in range(epochs):
    vgg16.train()  
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in trainloader:  

        optimiser.zero_grad()  
        outputs = vgg16(images)  
        loss = crit(outputs, labels)  

        losses.append(loss.item())  

        loss.backward()  
        optimiser.step()  

        running_loss += loss.item()
        _, predicted = outputs.max(1)  
        correct += predicted.eq(labels).sum().item()  
        total += labels.size(0)  

    train_acc = 100 * correct / total  

    if epoch % 10 == 0:  
        print(f'Epoch: {epoch}, Loss: {running_loss / len(trainloader):.4f}, Accuracy: {train_acc:.2f}%')
